In [18]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the dataset for recommenders

In [19]:
data_path = os.path.join("./data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df.loc[:, 'term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df.loc[:, 'length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df.loc[:, 'rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df.loc[:, 'room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df.loc[:, 'n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df.loc[:, 'weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df.loc[:, 'weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(HTML(interactions_df.head(15).to_html()))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several version of the prepare_users_df method and test which works best in your recommender.

In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

def prepare_users_df(interactions_df):

    # Write your code here
    interactions_categories = interactions_df.loc[:,['term','length_of_stay_bucket','rate_plan','room_segment','n_people_bucket','weekend_stay']]
    ohe = OneHotEncoder(handle_unknown='ignore')
    ohe.fit(interactions_categories)
    
    interactions_user_categories = interactions_df.loc[:,['user_id','term','length_of_stay_bucket','rate_plan','room_segment','n_people_bucket','weekend_stay']]

    user_occurences = interactions_user_categories.groupby("user_id").count().sort_values(by="user_id")

    

    all_cols = list(interactions_user_categories.columns)
    all_cols.remove("user_id")

    main_users_df = pd.DataFrame()
    for i,elem in enumerate(all_cols):
        tmp_interactions_categories = interactions_user_categories.loc[:,["user_id",elem]]
        tmp_interactions_categories.loc[:,"ones"]=1
        tmp_pivot = pd.pivot_table(tmp_interactions_categories,index="user_id",columns=elem,values="ones",aggfunc=np.sum,fill_value=0)

        tmp_pivot.columns=["uf"+str(i)+str(k) for k in range(len(column_values_dict.get(elem)))]

        if i==0:
            main_users_df=tmp_pivot
        else:
            main_users_df=pd.merge(main_users_df,tmp_pivot,on=["user_id"],how='left')
        
    main_users_df = main_users_df.apply(lambda x:x/user_occurences['term'])
    main_users_df=main_users_df.fillna(0.0)
        
    users_df = main_users_df.reset_index()

    user_features = list(users_df.columns)
    user_features.remove("user_id")
    
    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)

# print(user_features)
# display(HTML(users_df[users_df==np.NAN].head(20).to_html()))
display(HTML(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15).to_html()))

,user_id,uf00,uf01,uf02,uf03,uf04,uf05,uf06,uf07,uf10,uf11,uf12,uf13,uf20,uf21,uf30,uf31,uf32,uf33,uf34,uf40,uf41,uf42,uf43,uf50,uf51
0,1,0.130435,0.0,0.652174,0.086957,0.130435,0.000000,0.000000,0.000000,0.000000,0.608696,0.391304,0.000000,0.521739,0.478261,0.000000,0.869565,0.130435,0.0,0.0,0.000000,0.739130,0.173913,0.086957,0.782609,0.217391
47,50,0.043478,0.0,0.434783,0.304348,0.217391,0.000000,0.000000,0.000000,0.000000,0.913043,0.086957,0.000000,0.260870,0.739130,0.000000,0.608696,0.391304,0.0,0.0,0.000000,0.173913,0.521739,0.304348,0.782609,0.217391
92,96,0.083333,0.0,0.708333,0.125000,0.041667,0.041667,0.000000,0.000000,0.250000,0.666667,0.041667,0.041667,0.291667,0.708333,0.083333,0.791667,0.083333,0.0,0.0,0.041667,0.333333,0.541667,0.083333,0.750000,0.250000
111,115,0.727273,0.0,0.272727,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.363636,0.136364,0.000000,1.000000,0.000000,0.000000,0.818182,0.181818,0.0,0.0,0.818182,0.090909,0.045455,0.045455,0.363636,0.636364
675,706,0.091988,0.0,0.451039,0.189911,0.207715,0.038576,0.011869,0.008902,0.169139,0.459941,0.272997,0.097923,0.994065,0.005935,0.014837,0.851632,0.127596,0.0,0.0,0.041543,0.094955,0.738872,0.124629,0.676558,0.323442
1699,1736,0.034483,0.0,0.482759,0.206897,0.275862,0.000000,0.000000,0.000000,0.241379,0.551724,0.206897,0.000000,0.172414,0.827586,0.000000,0.931034,0.068966,0.0,0.0,0.379310,0.413793,0.206897,0.000000,0.448276,0.551724
7639,7779,0.037037,0.0,0.296296,0.259259,0.370370,0.000000,0.000000,0.037037,0.111111,0.296296,0.481481,0.111111,1.000000,0.000000,0.000000,0.888889,0.111111,0.0,0.0,0.000000,0.037037,0.740741,0.222222,0.814815,0.185185


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into on-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because of the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features.

In [22]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
def prepare_items_df(interactions_df):
    cols = ['item_id','term','length_of_stay_bucket','rate_plan','room_segment','n_people_bucket','weekend_stay']
    cats = []
    if 'if11' not in cols:
        cats = ['term','length_of_stay_bucket','rate_plan','room_segment','n_people_bucket','weekend_stay']
   
        interactions_item_categories = interactions_df.loc[:,cols]

        item_occurences = interactions_item_categories.groupby("item_id").count().sort_values(by="item_id")

        all_cols = list(interactions_item_categories.columns)
        all_cols.remove("item_id")

        main_items_df = pd.DataFrame()
        for i,elem in enumerate(all_cols):
            tmp_interactions_categories = interactions_item_categories.loc[:,["item_id",elem]]

            tmp_interactions_categories.loc[:,"ones"]=1
            tmp_pivot = pd.pivot_table(tmp_interactions_categories,index="item_id",columns=elem,values="ones",aggfunc=lambda x:1,fill_value=-1)
            
            if len(tmp_pivot.columns)!=len(column_values_dict.get(elem)):
                diff = set(column_values_dict.get(elem))-set(tmp_pivot.columns)
                for ii in list(diff):
                    tmp_pivot.loc[:,ii]=-1

            tmp_pivot.columns=["if"+str(i)+str(k) for k in range(len(column_values_dict.get(elem)))]

            if i==0:
                main_items_df=tmp_pivot
            else:
                main_items_df=pd.merge(main_items_df,tmp_pivot,on=["item_id"],how='left')

        items_df = main_items_df.reset_index()

    else:
        items_df = interactions_df
    items_df = items_df.fillna(-1.0)
    
    item_features = list(items_df.columns)
    item_features.remove("item_id")
    
    return items_df.loc[:,["item_id"]+item_features], item_features


items_df, item_features = prepare_items_df(interactions_df)

display(HTML(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15).to_html()))

,item_id,if00,if01,if02,if03,if04,if05,if06,if07,if10,if11,if12,if13,if20,if21,if30,if31,if32,if33,if34,if40,if41,if42,if43,if50,if51
0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1.0,-1.0,1.0,-1.0,-1.0,-1,-1,-1,1,1,-1
1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1.0,1.0,-1.0,-1.0,-1.0,-1,-1,1,-1,1,-1
2,2,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1.0,1.0,-1.0,-1.0,-1.0,-1,1,-1,-1,-1,1
3,3,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1.0,1.0,-1.0,-1.0,-1.0,-1,-1,1,-1,1,-1
4,4,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1,1,-1,-1,1,-1
5,5,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1.0,-1.0,1.0,-1.0,-1.0,-1,-1,-1,1,1,-1
6,6,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1.0,-1.0,1.0,-1.0,-1.0,-1,-1,-1,1,1,-1


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object for later use in the recommend method. Overwrite the users_df variable. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real interactions and equal to 0 for those newly added interactions. Generate several negative interactions per every positive interactions (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and use initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, Neural Networks or any model of your choice.
  - A different approach where you treat each item as a class, you train directly on categorical features of items and users (you would have to design appropriate categorical features for users) and you fit classifiers (e.g. Decision Tree classifier, Naive Bayes classifier etc.) instead of regressors.

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from functools import reduce
import math
from sklearn.preprocessing import StandardScaler

from recommenders.recommender import Recommender
from sklearn.feature_selection import SelectKBest, chi2, f_regression,f_classif


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
        self.model_features = None
        self.importance = None
        self.std = None
        self.fit_std = None
        self.final_std = None
        self.users_std = None
        self.items_std = None
    

    def get_best_features2(self,interactions_df,prod_features,k):
        selector = SelectKBest(f_classif, k=k)
        prod_features_df = interactions_df.loc[:,prod_features+["interacted"]]
        y = prod_features_df['interacted'].values

        X = prod_features_df.loc[:,prod_features].values

        selector.fit(X, y)
        
        cols = selector.get_support(indices=True)
        features_with_scores = sorted([("prod"+str(index),elem) for index,elem in enumerate(selector.scores_) if not math.isnan(elem)],key=lambda x:x[1],reverse=True)[:k]
        features_with_scores = {i:v for i,v in features_with_scores}
        total = sum(features_with_scores.values())
        self.importance = {k:float(v/total) for k,v in features_with_scores.items()}

        selected_features_df = prod_features_df.iloc[:,cols]
        selected_features = selected_features_df.columns.tolist()

        features_df_new = interactions_df.loc[:,selected_features+["interacted"]]

        self.model_features = selected_features
        return features_df_new
    
    def build_user_df(self,users_df):
        unique_users = users_df["user_id"].unique()
        
        new_users_df = pd.merge(users_df,self.users_df,on=['user_id'])
        unique_new_users = new_users_df["user_id"].unique()
        
        users_to_create = list(set(unique_new_users)-set(unique_users))
        new_df = pd.DataFrame({"user_id":users_to_create})
        new_df.loc[:,self.user_features]=0.0
        new_users_df = pd.concat([new_users_df,new_df])
        return new_users_df


    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        print("FIT")
        interactions_df = interactions_df.copy()

        users_df, user_features = prepare_users_df(interactions_df)
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        negative_interactions = []
        
        items_unique = interactions_df.loc[:,"item_id"].unique()
        users_unique = interactions_df.loc[:,"user_id"].unique()

        desire=int(self.n_neg_per_pos * len(interactions_df))
        hmany=100
        rng = np.random.RandomState(seed=6789)
        real_interaction = interactions_df.loc[:,["user_id","item_id"]]
        length = 0
        total_negative_interactions=pd.DataFrame(negative_interactions)
        while length!=desire and length<desire :
            it = rng.choice(items_unique,hmany)
            us = rng.choice(users_unique,hmany)
            tmp_negative_interaction = pd.DataFrame({'user_id':us,'item_id':it}).drop_duplicates()

            result = pd.merge(tmp_negative_interaction,real_interaction,on=["user_id","item_id"])
            if len(result)==0:
                total_negative_interactions = pd.concat([total_negative_interactions,tmp_negative_interaction]).drop_duplicates()
                length = total_negative_interactions.size

        total_negative_interactions = total_negative_interactions.iloc[:desire]
        total_negative_interactions.loc[:,'interacted']=0
        negative_interactions = total_negative_interactions.to_numpy()
        
        # Write your code here
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions 
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.
        
        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'],how='left')
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'],how='left')

        u_colls = users_df.columns.tolist()
        u_colls.remove("user_id")
        i_colls = items_df.columns.tolist()
        i_colls.remove("item_id")


        prod_cols=[]

        for index,elem in enumerate(zip(u_colls,i_colls)):

            tmp_result = interactions_df.loc[:,elem[0]] * interactions_df.loc[:,elem[1]]
            interactions_df.loc[:,"prod"+str(index)]=tmp_result
            prod_cols.append("prod"+str(index))

        interactions_df = interactions_df.fillna(0.0)
            
        new_interactions_df = self.get_best_features2(interactions_df,prod_cols,k=11)

        self.final_std = StandardScaler()
        scaled_df = pd.DataFrame(self.final_std.fit_transform(new_interactions_df.loc[:,self.model_features]),columns=self.model_features)
        
        x = scaled_df.loc[:,self.model_features].values
        y = new_interactions_df['interacted'].values
    
        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        print("RECOMMEND")
        self.recommender_df = self.recommender_df[:0]
        
        # Write your code here
        # Prepare users_df and items_df
        # For users_df you just need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations)
        # For items you have to apply the prepare_items_df method to items_df.
        
        users_df = self.build_user_df(users_df)

        items_df, item_features = prepare_items_df(items_df)
        
        items_df_tmp = items_df.copy()
        
        u_colls = users_df.columns.tolist()
        u_colls.remove("user_id")
        i_colls = items_df_tmp.columns.tolist()
        i_colls.remove("item_id")
    
        users_ids = users_df.loc[:,"user_id"]

        users_df.loc[:,"user_id"]=users_ids
        
        # Score the items
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        items_ids = items_df_tmp.loc[:,"item_id"]
        items_df_tmp.loc[:,"item_id"]=items_ids

        for ix, user in users_df.iterrows():

            prod_cols=[]
            
            # Write your code here
            # Create a Carthesian product of users from users_df and items from items_df
            tmp_user = pd.DataFrame(user).transpose()

            tmp_to_recommend = tmp_user.merge(items_df_tmp,how="cross").sort_values(by="item_id",ascending=True)
            tmp_to_recommend = tmp_to_recommend.drop(["user_id","item_id"],1)

            for index,elem in enumerate(zip(u_colls,i_colls)):
                tmp_result = tmp_to_recommend.loc[:,elem[0]] * tmp_to_recommend.loc[:,elem[1]]
                tmp_to_recommend.loc[:,"prod"+str(index)]=tmp_result
                prod_cols.append("prod"+str(index))
                
            
            tmp_to_recommend = tmp_to_recommend.fillna(0.0)
            
            tmp_to_recommend = tmp_to_recommend.loc[:,self.model_features]

            item_ids = items_df_tmp.loc[:,"item_id"]
            
            tmp_to_recommend = pd.DataFrame(self.final_std.transform(tmp_to_recommend.loc[:,self.model_features]),columns=self.model_features)

            tmp_recommended = pd.DataFrame(self.model.predict(tmp_to_recommend),columns=["score"])
            tmp_recommended.loc[:,"item_id"]=item_ids
            res = tmp_recommended
            
            res=res.sort_values(by=["score"],ascending=False)[:n_recommendations]
            
            
            # Write your code here
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
            chosen_ids = res.loc[:,"item_id"].to_numpy()
            scores = res.loc[:,"score"].to_numpy()
            
            recommendations = []
            for i,item_id in enumerate(chosen_ids):
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[i]
                    }
                )
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [24]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [25]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()

cb_user_item_recommender.fit(interactions_df, None, None)

FIT


/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [26]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)


recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(HTML(recommendations.to_html()))

RECOMMEND


,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,29,0.998417,OffSeason,[4-7],Standard,[160-260],[2-2],True
1,1.0,102,0.991514,OffSeason,[2-3],Nonref,[160-260],[2-2],True
2,1.0,41,0.986070,OffSeason,[2-3],Standard,[160-260],[2-2],True
3,1.0,31,0.978205,OffSeason,[4-7],Standard,[160-260],[2-2],False
4,1.0,26,0.968910,OffSeason,[2-3],Standard,[160-260],[2-2],False
5,1.0,100,0.961606,OffSeason,[2-3],Nonref,[160-260],[2-2],False
6,1.0,415,0.927830,OffSeason,[4-7],Standard,[260-360],[2-2],True
7,1.0,128,0.927830,OffSeason,[4-7],Standard,[0-160],[2-2],True
8,1.0,189,0.926586,LowSeason,[2-3],Nonref,[160-260],[2-2],True
9,1.0,206,0.923912,LowSeason,[2-3],Standard,[160-260],[2-2],True


TEST


# Tuning method

In [27]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [28]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)
    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(HTML(results.to_html()))
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [12]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

FIT                                                   
RECOMMEND                                             
  0%|          | 0/10 [00:02<?, ?trial/s, best loss=?]

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw



FIT                                                                                
RECOMMEND                                                                          
 10%|█         | 1/10 [00:34<04:39, 31.10s/trial, best loss: -0.039328144203195414]

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw



FIT                                                                                
RECOMMEND                                                                          
 20%|██        | 2/10 [01:05<04:13, 31.73s/trial, best loss: -0.039328144203195414]

...

100%|██████████| 10/10 [05:21<00:00, 32.12s/trial, best loss: -0.04096681687832855]
FIT


/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


RECOMMEND


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.010201,0.023034,0.028628,0.045081,0.010201,0.017436,0.019701,0.024932


Best parameters:
{'n_neg_per_pos': 3.0}


In [23]:
print("TEST")
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.1), np.log(10.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  f = msb / msw

RECOMMEND
FIT
 62%|██████▏   | 62/100 [1:59:39<45:57, 72.57s/trial, best loss: -0.04014748054076198]/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw

RECOMMEND
FIT
 63%|██████▎   | 63/100 [2:00:29<40:32, 65.74s/trial, best loss: -0.04014748054076198]/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value enc

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,SVRCBUIRecommender,0.010201,0.018756,0.027641,0.04179,0.010201,0.015297,0.01895,0.023479


Best parameters:
{'C': 0.10022460759031174, 'n_neg_per_pos': 3.0}


In [24]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

ction.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw

RECOMMEND
FIT
 62%|██████▏   | 62/100 [39:28<21:35, 34.11s/trial, best loss: -0.03441212617779599]/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw

RECOMMEND
FIT
 63%|██████▎   | 63/100 [40:03<21:14, 34.45s/trial, best loss: -0.03441212617779599]/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_select

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.007897,0.016453,0.025337,0.035538,0.007897,0.012821,0.016475,0.019718


Best parameters:
{'max_depth': 8.0, 'min_samples_split': 19.0, 'n_estimators': 289.0, 'n_neg_per_pos': 5.0}


In [25]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

7%|████████▋ | 262/300 [2:45:10<24:11, 38.19s/trial, best loss: -0.04219582138467841]/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw

RECOMMEND
FIT
 88%|████████▊ | 263/300 [2:45:51<23:59, 38.89s/trial, best loss: -0.04219582138467841]/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,

/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = m

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.009543,0.021718,0.032247,0.044423,0.009543,0.016707,0.021011,0.024917


Best parameters:
{'learning_rate': 0.028018695121238377, 'max_depth': 4.0, 'min_samples_split': 22.0, 'n_estimators': 205.0, 'n_neg_per_pos': 7.0}


# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [29]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(
    **{'n_neg_per_pos': 3})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(linear_cbui_tts_results.to_html()))

FIT


/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [17 18] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/home/rp/anaconda3/envs/REK/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


RECOMMEND


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.010201,0.023034,0.028628,0.045081,0.010201,0.017436,0.019701,0.024932


In [15]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(amazon_tts_results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.04179,0.105298,0.141494,0.200066,0.04179,0.07712,0.092218,0.110992


In [16]:
tts_results = pd.concat([linear_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(HTML(tts_results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.010201,0.023034,0.028628,0.045081,0.010201,0.017436,0.019701,0.024932
1,AmazonRecommender,0.041790,0.105298,0.141494,0.200066,0.041790,0.077120,0.092218,0.110992
